In [ ]:
import numpy as np
import pandas as pd
import random
import os
import logging
from collections import defaultdict
from tqdm import tqdm
import json
from datetime import datetime

In [8]:
class AntColonySupplyChainOptimizer:
    def __init__(self, demand_df, prices_df, max_load=8000, transport_cost=4500,
                 iterations=100, n_ants=20, evaporation_rate=0.1, alpha=1.0, beta=2.0):
        self.original_demand_df = demand_df.copy()
        self.prices_df = prices_df.copy()
        self.max_load = max_load
        self.transport_cost = transport_cost
        self.iterations = iterations
        self.n_ants = n_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha  # pheromone importance
        self.beta = beta    # heuristic importance

        self.demands = self.original_demand_df.groupby(['polygon', 'specie'])['demand'].sum().reset_index()
        self.species = self.demands['specie'].unique()
        self.suppliers = self.prices_df['supplier'].unique()

        # Pheromone initialization
        self.pheromones = defaultdict(lambda: 1.0)

        self.log_file = 'ACO_Supply_Chain.log'
        if os.path.exists(self.log_file):
            logging.shutdown()
            os.remove(self.log_file)
        logging.basicConfig(filename=self.log_file, level=logging.INFO)

    def heuristic(self, polygon, specie, supplier):
        row = self.prices_df[(self.prices_df['specie'] == specie) &
                             (self.prices_df['supplier'] == supplier)]
        if row.empty:
            return 1e-6  # discourage invalid supplier
        return 1.0 / row.iloc[0]['price']

    def construct_solution(self):
        remaining = self.demands.copy()
        schedule = []
        day = 0

        while remaining['demand'].sum() > 0:
            day += 1
            remaining_capacity = self.max_load
            fulfilled_indices = []
            day_plan = []

            for idx, row in remaining[remaining['demand'] > 0].sample(frac=1).iterrows():
                polygon, specie, demand = row['polygon'], row['specie'], row['demand']
                if demand > remaining_capacity:
                    continue

                valid_suppliers = self.prices_df[self.prices_df['specie'] == specie]['supplier'].unique()
                if len(valid_suppliers) == 0:
                    continue

                probs = []
                for supplier in valid_suppliers:
                    pheromone = self.pheromones[(polygon, specie, supplier)] ** self.alpha
                    heuristic_val = self.heuristic(polygon, specie, supplier) ** self.beta
                    probs.append(pheromone * heuristic_val)

                if sum(probs) == 0:
                    continue

                probs = np.array(probs)
                probs /= probs.sum()
                supplier = np.random.choice(valid_suppliers, p=probs)

                day_plan.append((day, polygon, specie, supplier, demand))
                remaining_capacity -= demand
                fulfilled_indices.append(idx)

            schedule.extend(day_plan)
            remaining.loc[fulfilled_indices, 'demand'] = 0

        return schedule

    def fitness(self, schedule):
        cost = 0
        horizon_days = max(entry[0] for entry in schedule)
        for day in range(1, horizon_days + 1):
            day_orders = [entry for entry in schedule if entry[0] == day]
            day_suppliers = set()
            for (_, polygon, specie, supplier, amount) in day_orders:
                row = self.prices_df[(self.prices_df['specie'] == specie) &
                                     (self.prices_df['supplier'] == supplier)]
                unit_price = row.iloc[0]['price']
                cost += unit_price * amount
                day_suppliers.add(supplier)
            cost += self.transport_cost * len(day_suppliers)
        return cost

    def update_pheromones(self, all_solutions):
        # Evaporation
        for key in self.pheromones:
            self.pheromones[key] *= (1 - self.evaporation_rate)

        # Reinforcement with best solution
        best_solution, best_cost = min(all_solutions, key=lambda x: x[1])
        for (_, polygon, specie, supplier, _) in best_solution:
            self.pheromones[(polygon, specie, supplier)] += 1.0 / best_cost

    def run(self):
        best_overall_solution = None
        best_overall_cost = float('inf')

        for it in tqdm(range(self.iterations), desc='ACO optimization'):
            solutions = []
            for _ in range(self.n_ants):
                candidate = self.construct_solution()
                cost = self.fitness(candidate)
                solutions.append((candidate, cost))

            iteration_best, iteration_cost = min(solutions, key=lambda x: x[1])
            if iteration_cost < best_overall_cost:
                best_overall_cost = iteration_cost
                best_overall_solution = iteration_best

            self.update_pheromones(solutions)

            logging.info(f"Iteration {it+1}: Best cost = {iteration_cost:.2f}")

        logging.info(f"Best solution found with cost: {best_overall_cost:.2f}")
        return best_overall_solution, best_overall_cost

In [ ]:
demand_df = pd.read_csv('../setup/demand.csv')
demand_df = demand_df.sample(frac=0.1)
prices_df = pd.read_csv('../setup/supplier_prices.csv')

In [10]:
demand_df

,specie,polygon,demand
0,Agave lechuguilla,1,178
1,Agave salmiana,1,847
2,Agave scabra,1,178
3,Agave striata,1,178
4,Opuntia cantabrigiensis,1,210
...,...,...,...
305,Opuntia engelmani,31,220
306,Opuntia robusta,31,425
307,Opuntia streptacanta,31,374
308,Prosopis laevigata,31,506


In [11]:
optimizer = AntColonySupplyChainOptimizer(demand_df, prices_df)
best_solution, best_cost = optimizer.run()

ACO optimization: 100%|██████████| 100/100 [24:45<00:00, 14.85s/it]


In [12]:
temp = pd.DataFrame(best_solution, columns = ['Day', 'Polygon', 'Specie', 'Supplier', 'Amount'])

orders = temp.groupby(['Day', 'Supplier', 'Specie']).agg({'Amount' : 'sum'}).reset_index()
vrp_nodes = temp.groupby('Day').agg({'Amount' : 'unique'}).reset_index()

orders_list = orders.to_dict(orient='records')
with open('../scheduling/ACO_orders.json', 'w') as f:
    json.dump(orders_list, f, indent=2)

In [13]:
filepath = '../scheduling/ACO_Supply_Chain_Solution.json'

# Convert numpy.float64 to float for JSON serialization
save_data = {
    'best_solution': best_solution,
    'best_cost': float(best_cost),
    'datetime': datetime.now().isoformat()
}

if not os.path.exists(filepath):
    with open(filepath, 'w') as f:
        json.dump(save_data, f, indent=2)
    print("Solution saved.")
else:
    with open(filepath, 'r') as f:
        existing_data = json.load(f)
    if float(best_cost) < float(existing_data['best_cost']):
        with open(filepath, 'w') as f:
            json.dump(save_data, f, indent=2)
        print("New best solution saved.")
    else:
        print("Existing solution is better or equal. No changes made.")

Solution saved.


In [19]:
demand_df

,specie,polygon,demand
0,Agave lechuguilla,1,178
1,Agave salmiana,1,847
2,Agave scabra,1,178
3,Agave striata,1,178
4,Opuntia cantabrigiensis,1,210
...,...,...,...
305,Opuntia engelmani,31,220
306,Opuntia robusta,31,425
307,Opuntia streptacanta,31,374
308,Prosopis laevigata,31,506


In [18]:
temp

,Day,Polygon,Specie,Supplier,Amount
0,1,7,Opuntia cantabrigiensis,Venado,245
1,1,4,Prosopis laevigata,Vivero,551
2,1,9,Opuntia cantabrigiensis,Moctezuma,312
3,1,24,Agave lechuguilla,Laguna seca,186
4,1,10,Opuntia robusta,Moctezuma,463
...,...,...,...,...,...
305,13,2,Prosopis laevigata,Vivero,518
306,13,5,Agave salmiana,Laguna seca,1185
307,13,19,Agave lechuguilla,Laguna seca,162
308,13,13,Agave scabra,Laguna seca,263
